# Parameters passed from Pipeline

In [25]:
dynamicyear = ""
dynamicmonth = ""
stgAccountName = ""
retentionDate = ""
retentionTimeInYears = 2

StatementMeta(sysparkpoolyd10, 89, 1, Finished, Available)

# Delete data no longer needed on the retention policy in the Data Lake (2 years in our example)

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

adls2_account_name = stgAccountName
adls2_container_name = "datalake"
adls2_folderyear = dynamicyear
adls2_foldermonth = dynamicmonth

spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false")  
delta_table_path = 'abfss://{0}@{1}.dfs.core.windows.net/{2}/{3}/'.format(adls2_container_name, adls2_account_name, adls2_folderyear, adls2_foldermonth)

# Create Delta Table Object
deltaTableLake = DeltaTable.forPath(spark, delta_table_path )

# Delete data according to the company police (2 years on our example)
deleteStatement = '{0}'.format(retentionDate)
deltaTableLake.delete("tpep_pickup_datetime < '{0}'".format(deleteStatement)) 
# Vacuum files that will no longer be referenced
spark.sql("SET spark.databricks.delta.retentionDurationCheck.enabled = false") 
# zero refers to no retention time for this files, so it means they will be deleted immediately
deltaTableLake.vacuum(0)



# Update configuration file (deleting data no covered by retention, so is not re-ingested)

## Import Libraries

In [32]:
import json
from notebookutils import mssparkutils
from datetime import datetime
from adlsaccess.synadlsaccess import ADLSOps

StatementMeta(sysparkpoolyd10, 89, 8, Finished, Available)

## Define constants, variables and fetch secret values

In [27]:
# Constants
keyvault_ls_name = "Ls_NYCTaxi_KeyVault"
storage_key_name = "datalakeKey"
data_container = "datalake"
config_container = "config"
config_file_name = "datalake_config.json"
config_file_path = "/"

# Variables
current_year = datetime.utcnow().strftime("%Y")
current_month = int(datetime.utcnow().strftime("%m"))
retention_year = int(current_year)-int(retentionTimeInYears)
current_ts = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

# Secrets based values
storage_access_key = mssparkutils.credentials.getSecretWithLS(keyvault_ls_name, storage_key_name)
storage_acct_connection = f"DefaultEndpointsProtocol=https;AccountName={stgAccountName};AccountKey={storage_access_key};EndpointSuffix=core.windows.net"


StatementMeta(sysparkpoolyd10, 89, 3, Finished, Available)

## Get configuration file, and delete entries that are no longer in the retention period

In [ ]:
adls_ops = ADLSOps(stgAccountName, keyvault_ls_name, storage_key_name)
adls_ops.initialize_storage_account(storage_access_key)
config = adls_ops.read_config_from_adls()
configDeleteFinal=adls_ops.update_datalake_config_with_retention(config,retention_year,current_month)

# mssparkutils.fs.help()
source_config = f"abfss://{adls_ops.config_container}@{stgAccountName}.dfs.core.windows.net{adls_ops.config_file_path}{adls_ops.config_file_name}"
backup_config = f"abfss://{adls_ops.config_container}@{stgAccountName}.dfs.core.windows.net{adls_ops.config_file_path}{adls_ops.config_file_name}_{current_ts}"

mssparkutils.fs.mv(source_config, backup_config, overwrite=True)
jsonResult=json.dumps(configDeleteFinal,indent=2, default=str)   
mssparkutils.fs.put(source_config, jsonResult, overwrite=True)
